# システム統合１（有限オートマトン）
それでは、これまでに実装してきたモジュールを統合して音声対話システムを作成してみましょう。

- 仕様
    - 音声認識：Google（ストリーミング、オリジナルVAD）
    - 音声合成：Google
    - 言語理解：ルールベース（フレーム）
    - 対話管理：有限オートマトン

In [5]:
# 必要なラブラリを読み込む

# 実装したライブラリを読み込む
from asr_google_streaming_vad import GoogleStreamingASR, MicrophoneStream   # VADあり
from tts_google import GoogleTextToSpeech
from my_dm_fst import DmFst
from my_slu_rule import SluRule
from my_ocr import OCR
from pathlib import Path

In [6]:
# 入力した画像に対してOCRを行う
recipe_path = "./recipes/recipe6.png"
ocr = OCR(Path(recipe_path) if recipe_path else None)
procedures = ocr.detect_chars()

In [7]:
from pprint import pprint
pprint(procedures)

{1: '1番．．．ニニ杯酢を作る。酢を耐熱容器に入れ、電子レンジに20秒かけ、酢気をとばす。冷ましてからしょうゆを合わせる。',
 2: '2番．．．あじの両面に薄く塩をし、15-~20分おく。',
 3: '3番．．．バットに酢を入れ、②を入れて5分浸ける。小骨を抜く。',
 4: '4番．．．あじの頭側から皮をむく。まず、指先で皮をめくる。',
 5: '5番．．．少しむいたら、もう一方の手で身を押さえながら静かにむいていく。',
 6: '6番．．．銀皮側に鹿の子に切り目を入れ、ひと口大に切って器に盛る。きゅうり、わかめを盛り合わせ、おろししょうがをのせて二杯酢をかける。．．これでレシピは以上です．'}


In [8]:
# 音声認識クラスのパラメータ
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

# 音声合成の初期化
tts = GoogleTextToSpeech()

# 言語理解の初期化
slu_parser = SluRule(num_proc=len(procedures))

In [9]:
# 対話管理の初期化
dm = DmFst(recipe=procedures)

# 初期状態の発話
system_utterance = dm.get_system_utterance()
tts.generate(system_utterance)
print("システム： " + system_utterance)
tts.play()

# 対話が終了状態に移るまで対話を続ける
while(dm.end == False):

    # 音声認識入力を得る
    micStream = MicrophoneStream(RATE, CHUNK)
    asrStream = GoogleStreamingASR(RATE, micStream)
    print('<<<please speak>>>')
    result_asr = asrStream.get_asr_result()

    if hasattr(result_asr, 'alternatives') == False:
        print('Invalid ASR input')
        continue

    result_asr_utterance = result_asr.alternatives[0].transcript
    print("ユーザ： " + result_asr_utterance)
    
    # 言語理解
    result_slu = slu_parser.parse_frame(result_asr_utterance)
    print(result_slu)
    
    # 対話管理へ入力
    system_utterance = dm.enter(result_slu)
    tts.generate(system_utterance)
    print("システム： " + system_utterance)
    tts.play()
    
    print()


システム： レシピ読み上げさんです．レシピ写真をアップロードしてください．
<<<please speak>>>
音声パワー： 45.1[dB] 

KeyboardInterrupt: 

音声パワー： 28.1[dB] 